# Benchmark

Whenever we make changes to the code we will rerun a simple benchmark in order to keep track of how the simulation results might change over time. It is in general hard to verify that the equations are implemented correctly because there exist no analytic solutions to the equations. However, by running the same benchmark over time we can see if the changes we make will impact the simulation results. Here the different biomarkers are taken from traces evaluated a the center of at slab that is simulated for 1000 ms (see the benchmark in the library for more details)

In [ ]:
import json
import numpy as np
from collections import defaultdict
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import requests

pd.options.display.max_columns = 5

In [ ]:
headers = {
    "Accept": "application/vnd.github+json",
    "X-GitHub-Api-Version": "2022-11-28",
}
print("Get existing data from gist")
response = requests.get(
    "https://api.github.com/gists/73fa6531f28da2b3633a7ddaca38a7cd",
    headers=headers,
)
data = json.loads(response.json()["files"]["convergence_test.json"]["content"])

You can checkout a specific commit by going to the link

https://github.com/ComputationalPhysiology/simcardems/commit/GIT_COMMIT_HASH

where you swap out GIT_COMMIT_HASH with the specific hash, i.e

In [ ]:
for sha in data.keys():
    print(f"https://github.com/ComputationalPhysiology/simcardems/commit/{sha}")

In [ ]:
len(data)

In [ ]:
excluded_keys = ["dt", "dx", "sha", "num_cells_mechanics", "num_cells_ep", "num_vertices_mechanics", "num_vertices_ep"]
all_results = defaultdict(list)
for git_hash, d in data.items():
    all_results["git_hash"].append(git_hash)
    for key, value in d.get("dx0.2_dt0.05").items():
        if key in excluded_keys:
            continue
        all_results[key].append(value)

results = {}
for k, v in all_results.items():
    print(k, len(v))
    if len(v) == len(data):
        results[k] = v

In [ ]:
df = pd.DataFrame(results)
df["timestamp"] = pd.to_datetime(df["timestamp"])
df

In [ ]:
df = df.sort_values(by="timestamp")
df

In [ ]:
df[["simcardems_version", "timestamp", "git_hash"]]

In [ ]:
def get_ylim(values):
    if np.isclose(values, 0).all():
        return (-0.01, 0.01)
    y_mean = np.mean(values)
    y_max = max(np.max(values), y_mean + 0.01 * abs(y_mean))
    y_min = min(np.min(values), y_mean - 0.01 * abs(y_mean))
    return (y_min, y_max)

In [ ]:
columns = [c for c in df.columns if c not in ["timestamp", "simcardems_version", "git_hash"]]
git_hash = df["git_hash"]
versions = df["simcardems_version"]
dates = [t.date() for t in df["timestamp"]]

text = []
for h, v, t in zip(git_hash, versions, dates):
    text.append(
        "\n".join(
            [
                f"<br>Git Hash: {h}</br>",
                f"<br>Version: {v}</br>",
                f"<br>Timestamp {t}</br>",
            ]
        )
    )

fig = make_subplots(rows=len(columns), cols=1, subplot_titles=columns, shared_xaxes=True)

yranges = []
for i, c in enumerate(columns):
    row = i + 1
    col = 1

    fig.add_trace(go.Scatter(x=df["timestamp"], y=df[c], text=text, hovertemplate="%{text}"), row=row, col=col)
    fig.update_yaxes(range=get_ylim(df[c]), row=row, col=col)

fig.update_layout(height=5000, showlegend=False)
fig.show()